In [1]:
!pip install sentence-transformers chromadb requests scikit-learn



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
ticket = {
    "id": "TICK1234",
    "text": "I ordered a phone two weeks ago but it hasn’t arrived yet. What’s going on?",
    "customer_id": "USER987",
    "submitted_at": "2025-07-05"
}


In [5]:
import re
import requests
import chromadb
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [6]:
# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

In [7]:
# Categories for classification
CATEGORIES = [
    "Shipping Issue", "Return Request", "Payment Problem",
    "Product Quality", "Account/Login", "Technical Support", "General Inquiry"
]
category_embeddings = model.encode(CATEGORIES)

In [9]:
# Sample ticket
TICKET = {
    "id": "TICK1234",
    "text": "I ordered a phone two weeks ago but it hasn’t arrived yet. What’s going on?",
    "customer_id": "USER987",
    "submitted_at": "2025-07-05"
}

# Sample historical tickets and docs
HISTORICAL_TICKETS = [
    {"text": "My order hasn't arrived in 10 days", "solution": "Your package is delayed due to weather."},
    {"text": "Payment failed but amount was deducted", "solution": "Check your bank, refund initiated."},
    {"text": "How do I return a product?", "solution": "Please follow the return steps at company.com/returns"},
]

COMPANY_DOCS = [
    {"title": "Shipping Policy", "content": "We ship within 5-7 business days using courier partners."},
    {"title": "Return Policy", "content": "Returns are accepted within 30 days of purchase."}
]

In [11]:
from dotenv import load_dotenv
load_dotenv()
import os
# Groq API key (replace with your actual key)
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

In [12]:
# Step 1: Preprocessing
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

In [13]:
# Step 2: Categorization
def auto_categorize(ticket_text):
    ticket_emb = model.encode([ticket_text])[0]
    sims = cosine_similarity([ticket_emb], category_embeddings)[0]
    best_idx = sims.argmax()
    return CATEGORIES[best_idx], sims[best_idx]

In [14]:
# Step 3: Store tickets/docs in vector DB
def store_knowledge(tickets, docs):
    client = chromadb.Client()
    collection = client.create_collection("support_knowledge")
    for i, t in enumerate(tickets):
        collection.add(
            documents=[t["text"] + " " + t["solution"]],
            ids=[f"ticket_{i}"],
            metadatas=[{"type": "ticket", "source": t["text"]}]
        )
    for i, d in enumerate(docs):
        collection.add(
            documents=[d["content"]],
            ids=[f"doc_{i}"],
            metadatas=[{"type": "doc", "source": d["title"]}]
        )
    return collection

In [15]:
# Step 4: Retrieval
def retrieve_similar(ticket_text, collection, top_k=3):
    query_emb = model.encode([ticket_text])[0]
    result = collection.query(query_embeddings=[query_emb.tolist()], n_results=top_k)
    docs = result['documents'][0]
    metas = result['metadatas'][0]
    return [{"text": d, "source": m["source"], "type": m["type"]} for d, m in zip(docs, metas)]

In [16]:
# Step 5: Generate LLM response with Groq
def generate_response_groq(ticket_text, retrieved_docs):
    context = "\n\n".join([doc["text"] for doc in retrieved_docs])
    sources = "\n".join([f"- From {doc['type']}: {doc['source']}" for doc in retrieved_docs])

    messages = [
        {"role": "system", "content": "You are a smart support assistant. Use past ticket solutions and company docs to answer the new ticket."},
        {"role": "user", "content": f"""Ticket:
{ticket_text}

Relevant Knowledge:
{context}

Sources:
{sources}

Please generate a helpful, concise support response."""}
    ]

    response = requests.post(
        "https://api.groq.com/openai/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {GROQ_API_KEY}",
            "Content-Type": "application/json"
        },
        json={
            "model": "llama3-70b-8192",
            "messages": messages,
            "temperature": 0.3
        }
    )
    return response.json()['choices'][0]['message']['content']

In [17]:
# Step 6: Escalation check
def should_escalate(similarity_score, threshold=0.75):
    return similarity_score < threshold

In [18]:
ticket_clean = clean_text(ticket['text'])

In [20]:
category, confidence = auto_categorize(ticket_clean)
print(f"\nCategory: {category} (Confidence: {confidence:.2f})")


Category: Shipping Issue (Confidence: 0.46)


In [22]:
collection = store_knowledge(HISTORICAL_TICKETS, COMPANY_DOCS)
retrieved_docs = retrieve_similar(ticket_clean, collection)

C:\Users\harsh\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████| 79.3M/79.3M [01:51<00:00, 748kiB/s]


In [23]:
retrieved_docs

[{'text': "My order hasn't arrived in 10 days Your package is delayed due to weather.",
  'source': "My order hasn't arrived in 10 days",
  'type': 'ticket'},
 {'text': 'We ship within 5-7 business days using courier partners.',
  'source': 'Shipping Policy',
  'type': 'doc'},
 {'text': 'Returns are accepted within 30 days of purchase.',
  'source': 'Return Policy',
  'type': 'doc'}]

In [25]:
response = generate_response_groq(ticket_clean, retrieved_docs)
print("\nAI Response:\n", response)


AI Response:
 Hi there!

I apologize for the delay in receiving your phone order. Since you placed your order two weeks ago, I'd like to investigate this further. According to our shipping policy, we ship within 5-7 business days using our courier partners. It's possible that your package may be delayed due to weather conditions.

Can you please provide me with your order number so I can look into this further and provide you with an update on the status of your shipment? I'll do my best to get your phone to you as soon as possible.

Thank you for your patience and understanding.


In [26]:
if should_escalate(confidence):
        print("\n⚠️ Escalation required due to low confidence.")
else:
    print("\n✅ Handled automatically.")


⚠️ Escalation required due to low confidence.
